In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import jaydebeapi as dbdriver
import credential  # Ensure this contains your credentials like db_user, db_password
import pandas as pd
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from bs4 import BeautifulSoup
import re
from socket import gethostname
import os
from googleapiclient.errors import HttpError
import json
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd

In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/spreadsheets']

# Authenticate to Google API
def authenticate():
    credentials = service_account.Credentials.from_service_account_file(
        'service_account.json', scopes=SCOPES)
    return credentials

In [3]:
def update_google_doc(doc_id, content):
    try:
        creds = authenticate()
        doc_service = build('docs', 'v1', credentials=creds)
        
        doc = doc_service.documents().get(documentId=doc_id).execute()
        end_index = doc.get('body').get('content')[-1].get('endIndex')
        requests = [
            {
                'insertText': {
                    'location': {'index':  end_index-1 },
                    'text': content
                }
            }
        ]
        
        doc_service.documents().batchUpdate(
            documentId=doc_id,
            body={'requests': requests}
        ).execute()
        
        return True
    except HttpError as error:
        print(f"An error occurred while updating Google Doc: {error}")
        return False

In [4]:
def load_tracking_dict_from_spreadsheet(spreadsheet_title, folder_id=None):
    try:
        # Authenticate and connect to Google Sheets
        creds = authenticate()
        gc = gspread.authorize(creds)

        # Search for the spreadsheet in the specified folder
        if folder_id:
            drive_service = build('drive', 'v3', credentials=creds)
            query = f"'{folder_id}' in parents and name = '{spreadsheet_title}' and mimeType = 'application/vnd.google-apps.spreadsheet'"
            results = drive_service.files().list(
    q=query,
    spaces='drive',
    fields='files(id, name)',
    supportsAllDrives=True,
    includeItemsFromAllDrives=True
).execute()
            files = results.get('files', [])

            if not files:
                print(f"Spreadsheet '{spreadsheet_title}' not found in folder ID {folder_id}.")
                return {}

            # Get the file ID of the spreadsheet
            file_id = files[0]['id']
            spreadsheet = gc.open_by_key(file_id)
        else:
            # Open the spreadsheet by its title (assumes it's in "My Drive")
            spreadsheet = gc.open(spreadsheet_title)

        # Load data from the spreadsheet into a DataFrame
        sheet = spreadsheet.sheet1
        df = get_as_dataframe(sheet, dtype=str, na_values=[]).dropna(how='all')  # Clean empty rows
        df.set_index('Article ID', inplace=True)

        # Convert the DataFrame to a dictionary
        tracking_dict = df.to_dict(orient='index')
        return tracking_dict
    except Exception as e:
        print(f"An error occurred while loading the tracking dictionary: {e}")
        return {}

In [5]:
def clean_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text(separator=' ', strip=True)

In [6]:
df = pd.read_csv('subset_sampled_clusters.csv')
spreadsheet_folder_id = '0ADjQe-gJ6HwnUk9PVA'

In [7]:
df.shape

(11766, 7)

In [8]:
group_sizes = df.groupby(['Knowledge Base Article', 'Knowledge Base Article Links']).size()

# Step 2: Get only groups with size > 50
large_groups = group_sizes[group_sizes > 100].index
print(f"Number of groups with size > 100: {len(large_groups)}")
large_groups_df = df[df.set_index(['Knowledge Base Article', 'Knowledge Base Article Links']).index.isin(large_groups)]
large_groups_df

Number of groups with size > 100: 2


,Ticket ID,Title,Description,Knowledge Base Article,KB Article ID,Knowledge Base Article Links,Cluster
30,7162364,TeamDynamix Access,New employee. Need access to Teamdynamix tickets,"TeamDynamix Admins: Requesting Access, Approva...",3189,https://teamdynamix.umich.edu/TDClient/30/Port...,-1
57,7754376,TeamDynamix Access,CoE Accessibility technicians,"TeamDynamix Admins: Requesting Access, Approva...",3189,https://teamdynamix.umich.edu/TDClient/30/Port...,-1
75,8031925,TeamDynamix Access,These students are working on a project and wi...,"TeamDynamix Admins: Requesting Access, Approva...",3189,https://teamdynamix.umich.edu/TDClient/30/Port...,-1
80,7048603,TeamDynamix Groups,This is a TeamDynamix Groups Request. Please r...,Request a TeamDynamix Responsible Group,3131,https://teamdynamix.umich.edu/TDClient/30/Port...,-1
88,6383577,TeamDynamix Access,Student worker and forgot to give KB Access.&n...,"TeamDynamix Admins: Requesting Access, Approva...",3189,https://teamdynamix.umich.edu/TDClient/30/Port...,-1
...,...,...,...,...,...,...,...
11623,7203730,TeamDynamix Access,"Hi,<br><br>Please add Kristen McCray (mccrayk)...","TeamDynamix Admins: Requesting Access, Approva...",3189,https://teamdynamix.umich.edu/TDClient/30/Port...,-1
11650,7192767,TeamDynamix Groups,This is a TeamDynamix Groups Request. Please r...,Request a TeamDynamix Responsible Group,3131,https://teamdynamix.umich.edu/TDClient/30/Port...,-1
11660,8105078,TeamDynamix Access,Please provide access so person can submit SOD...,"TeamDynamix Admins: Requesting Access, Approva...",3189,https://teamdynamix.umich.edu/TDClient/30/Port...,-1
11684,6891766,TeamDynamix Access,<h1>Onboarding Request for eivan</h1>,"TeamDynamix Admins: Requesting Access, Approva...",3189,https://teamdynamix.umich.edu/TDClient/30/Port...,-1


In [9]:
tracking_dict_for_public = load_tracking_dict_from_spreadsheet("Public Tracking", spreadsheet_folder_id)
tracking_dict_for_um_login = load_tracking_dict_from_spreadsheet("UM-Login Tracking", spreadsheet_folder_id)
tracking_dict_for_support_staff = load_tracking_dict_from_spreadsheet("Support Staff Tracking", spreadsheet_folder_id)

In [10]:
# size of the tracking dictionaries
tracking_dict_for_um_login_size = len(tracking_dict_for_um_login)
tracking_dict_for_public_size = len(tracking_dict_for_public)
tracking_dict_for_support_staff_size = len(tracking_dict_for_support_staff)
print(f"Size of tracking dictionary for UM-Login: {tracking_dict_for_um_login_size}")
print(f"Size of tracking dictionary for Public: {tracking_dict_for_public_size}")
print(f"Size of tracking dictionary for Support Staff: {tracking_dict_for_support_staff_size}")

Size of tracking dictionary for UM-Login: 246
Size of tracking dictionary for Public: 1746
Size of tracking dictionary for Support Staff: 3086


In [11]:
tracking_sources = [tracking_dict_for_public, tracking_dict_for_um_login, tracking_dict_for_support_staff]

In [12]:
for (article, link), group in df.groupby(['Knowledge Base Article', 'Knowledge Base Article Links']):
    match = re.search(r'ID=([\w-]+)', link)
    if not match:
        print(f"Skipping article without valid ID: {article} | Link: {link}")
        continue

    article_id = match.group(1)

    doc_id = None
    for tracking_dict in tracking_sources:
        if article_id in tracking_dict:
            doc_id = tracking_dict[article_id]['doc_id']
            print(f"Found article ID '{article_id}' in tracking dict. Using Google Doc ID: {doc_id}")
            break

    if not doc_id:
        #print(f"Article ID '{article_id}' not found in any tracking dict. Skipping: {article}")
        continue

    # Compose content to append
    content_lines = [f"\nExample Queries that use this article:\n"]
    for _, row in group.iterrows():
        content_lines.append(f"- Title: {row['Title']}\n")
        desc_raw = row['Description']
        Clean_Description = clean_html(desc_raw) if pd.notna(desc_raw) else "  "

        content_lines.append(f"  Description: {Clean_Description}\n")
    content = ''.join(content_lines)
    try:
        update_google_doc(doc_id, content)
        print(f"Successfully updated Google Doc (ID: {doc_id}) for article '{article}'")
    except Exception as e:
        print(f"Failed to update Google Doc (ID: {doc_id}) for article '{article}': {e}")


Found article ID '8621' in tracking dict. Using Google Doc ID: 1xjr9von22hIMunIzHSIouLo7NAvvj7ikIAyDkjnlSG8
Successfully updated Google Doc (ID: 1xjr9von22hIMunIzHSIouLo7NAvvj7ikIAyDkjnlSG8) for article '"400 Bad Request" Error When Adding Members To MCommunity Group'
Found article ID '6468' in tracking dict. Using Google Doc ID: 1J58PrFxy7vRfD7x6bRQyZEoZBjJVxY-TmNldAnSsyN4
Successfully updated Google Doc (ID: 1J58PrFxy7vRfD7x6bRQyZEoZBjJVxY-TmNldAnSsyN4) for article '"Account Lockout!" - Account Recovery Locked Due to Repeated Reset Attempts'
Found article ID '7030' in tracking dict. Using Google Doc ID: 1FegVL7P7ukcwQ1sZH-PvllONclgenR6mBWozWcgiwI4
Successfully updated Google Doc (ID: 1FegVL7P7ukcwQ1sZH-PvllONclgenR6mBWozWcgiwI4) for article '"Accounts for People Who Leave U-M" Documentation'
Found article ID '9489' in tracking dict. Using Google Doc ID: 1XOta4z0o8BF9g-J7J_zZ0eu1lJlEv4AVEwcVJh0LOWI
Successfully updated Google Doc (ID: 1XOta4z0o8BF9g-J7J_zZ0eu1lJlEv4AVEwcVJh0LOWI) for 